In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time, datetime 

In [6]:

pd.set_option('display.max_columns', None)

# 定义数据库连接

In [7]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [8]:
sql1 = '''
SELECT 
tt.order_number '订单号', tt.true_name '姓名', tt.user_mobile '手机号', tt.product_name '产品名称', tt.create_time '下单日期', tt.refund_date '预计还款日期', tt.reality_refund_date '实际还款日期', 
tt.money '应付金额', tt.real_pay_money '实付金额', tt.min_date, tt.max_date, tt.diff_date
FROM (
SELECT  t.order_number,t.true_name,t.user_mobile,t.product_name,t.create_time, t.refund_date,t.reality_refund_date,t.money,t.real_pay_money,t.min_date,
case when max_date>=DATE_ADD(CURRENT_DATE, INTERVAL -1 DAY) then CURRENT_DATE else t.max_date END max_date,
case when t.min_date=t.max_date then DATEDIFF(CURRENT_DATE, t.min_date) 
			when max_date>CURRENT_DATE then DATEDIFF(CURRENT_DATE, t.min_date) 
			else DATEDIFF(t.max_date, t.min_date) END diff_date
FROM (
SELECT -- 查询总体订单和去重以及剔除商家
om.order_number,tmu.true_name,om.user_mobile,tod.product_name,om.create_time,tos.refund_date,tos.reality_refund_date,tos.money,tos.real_pay_money
,min(tos.refund_date)over(partition by om.order_number) min_date
,max(tos.refund_date)over(partition by om.order_number) max_date
FROM db_digua_business.t_postlease_receivables_monitoring   tprm 
LEFT JOIN db_digua_business.t_order om on tprm.order_id = om.id 
LEFT JOIN db_digua_business.t_merchant tmer on tmer.id = tprm.merchant_id
LEFT JOIN db_digua_business.t_order_stages tos on om.id=tos.order_id
-- LEFT JOIN db_digua_business.t_order_relet tor on om.id=tor.order_id
LEFT JOIN db_digua_business.t_order_details tod on tod.order_id=om.id 
LEFT JOIN db_digua_business.t_member_user tmu on tmu.mobile=om.user_mobile
-- LEFT JOIN db_digua_business.t_order_relet tor on tor.order_id=om.id
WHERE om.`status` in (2, 3, 4) 
AND tmer.shop_type !=2  -- 剔除企业 1、自营店铺；2、入驻店铺；3、代运营店铺'
AND tprm.merchant_id  not in (15,99)  -- 剔除CPS渠道合作 汇客好租
AND tprm.model_number not like '%探路者%'
-- AND tos.refund_date = DATE_ADD(CURRENT_DATE,INTERVAL +1 DAY)
AND tos.real_pay_money<tos.money
) t
) tt
WHERE diff_date<=0 and refund_date>=CURRENT_DATE and refund_date<=DATE_ADD(CURRENT_DATE, INTERVAL +1 DAY)

'''
df_order = query(sql1)
df_order.shape

(696, 12)

In [9]:
sql2 = '''
SELECT tt.order_number '订单号', tt.true_name '姓名', tt.user_mobile '手机号', tt.product_name '产品名称', tt.create_time '下单日期', tt.relet_start_time '预计还款日期', tt.money '应付金额', tt.min_time FROM (
SELECT t.order_number, t.true_name, t.user_mobile, t.product_name, t.create_time, t.relet_start_time, t.money,
min(relet_start_time)over(partition by t.order_id) min_time FROM(
SELECT om.order_number, tor.order_id, tmu.true_name, om.user_mobile, tod.product_name, om.create_time, tor.relet_start_time, tor.money FROM db_digua_business.t_postlease_receivables_monitoring   tprm
LEFT JOIN db_digua_business.t_order om on tprm.order_id = om.id 
LEFT JOIN db_digua_business.t_merchant tmer on tmer.id = tprm.merchant_id
LEFT JOIN db_digua_business.t_order_relet tor on tprm.order_id=tor.order_id
LEFT JOIN db_digua_business.t_order_details tod on tod.order_id=om.id 
LEFT JOIN db_digua_business.t_member_user tmu on tmu.mobile=om.user_mobile
WHERE tmer.shop_type !=2  -- 剔除企业 1、自营店铺；2、入驻店铺；3、代运营店铺'
AND tprm.merchant_id  not in (15,99)  -- 剔除CPS渠道合作 汇客好租
AND tprm.model_number not like '%探路者%'
and cycle_status = 1
) t
) tt
WHERE min_time in (CURRENT_DATE, DATE_ADD(CURRENT_DATE, INTERVAL +1 DAY)) and relet_start_time in (CURRENT_DATE, DATE_ADD(CURRENT_DATE, INTERVAL +1 DAY))
'''
df_order2 = query(sql2)
df_order2.shape

(127, 8)

In [10]:
sql3 = '''
SELECT yoo.order_number '订单号', tmu.true_name '姓名', om.user_mobile '手机号', tod.product_name '产品名称', om.create_time '下单日期', yoo.overdue_time '预计还款日期', yoo.overdue_money '应付金额'
FROM db_digua_business.t_order om
LEFT JOIN db_rent.ya_order_overdue yoo on yoo.order_id=om.id
LEFT JOIN db_digua_business.t_order_details tod on tod.order_id=om.id 
LEFT JOIN db_digua_business.t_member_user tmu on tmu.mobile=om.user_mobile
WHERE DATE_FORMAT(om.create_time,'%Y-%m')=DATE_FORMAT(DATE_ADD(CURRENT_DATE,INTERVAL -1 MONTH), '%Y-%m')  and yoo.type = 0
'''
df_order3 = query(sql3)
df_order3.shape

(273, 7)

In [11]:
sql4 = '''
SELECT  t.channel_name ,t.activity_name, 
t.order_number '订单号', t.true_name '姓名', t.user_mobile '手机号', t.product_name '产品名称', t.create_time '下单日期', t.refund_date '预计还款日期', t.reality_refund_date '实际还款日期', 
t.money '应付金额', t.real_pay_money '实付金额', diff_date '逾期天数',t.sort,t.order_method
FROM (
SELECT -- 查询总体订单和去重以及剔除商家
cc.`name` as channel_name
,pa.`name` as activity_name
,om.order_number,tmu.true_name,om.user_mobile,tod.product_name,om.create_time,tos.refund_date,tos.reality_refund_date,tos.money,tos.real_pay_money, tos.sort,
case 	when tos.reality_refund_date is null then DATEDIFF(CURRENT_DATE, tos.refund_date)
			when DATE_FORMAT(tos.reality_refund_date,'%Y-%m-%d')>DATE_FORMAT(tos.refund_date,'%Y-%m-%d') then DATEDIFF(tos.reality_refund_date,tos.refund_date)
			else null END diff_date
,om.order_method
FROM db_digua_business.t_postlease_receivables_monitoring   tprm 
LEFT JOIN db_digua_business.t_order om on tprm.order_id = om.id 
LEFT JOIN db_digua_business.t_merchant tmer on tmer.id = tprm.merchant_id
LEFT JOIN db_digua_business.t_order_stages tos on om.id=tos.order_id
-- LEFT JOIN db_digua_business.t_order_relet tor on om.id=tor.order_id
LEFT JOIN db_digua_business.t_order_details tod on tod.order_id=om.id 
LEFT JOIN db_digua_business.t_member_user tmu on tmu.mobile=om.user_mobile
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
WHERE om.`status` in (2, 3, 4) 
AND tmer.shop_type !=2  -- 剔除企业 1、自营店铺；2、入驻店铺；3、代运营店铺'
AND tprm.merchant_id  not in (15,99)  -- 剔除CPS渠道合作 汇客好租
AND tprm.model_number not like '%探路者%'
-- AND tos.refund_date = DATE_ADD(CURRENT_DATE,INTERVAL +1 DAY)
-- AND tos.real_pay_money<tos.money
AND DATE_FORMAT(om.create_time,'%Y-%m')=DATE_FORMAT(DATE_ADD(CURRENT_DATE, INTERVAL -1 MONTH), '%Y-%m')
-- AND DATE_FORMAT(tos.refund_date, '%Y-%m')=DATE_FORMAT(CURRENT_DATE,'%Y-%m')
-- AND tos.reality_refund_date>tos.refund_date
-- OR tos.reality_refund_date is NULL
AND tos.refund_date <= CURRENT_DATE
) t
WHERE DATE_FORMAT(t.refund_date,'%Y-%m')=DATE_FORMAT(CURRENT_DATE,'%Y-%m')
'''
df_order4 = query(sql4)
df_order4.shape

(5119, 14)

# 合并数据

In [12]:
df = df_order.copy()
df2 = df_order2.copy()
df3 = df_order3.copy()
df4 = df_order4.copy()
# df.order_id.notnull().sum()

# 渠道归属

In [13]:
df4.loc[:,"来源渠道"]=df4["channel_name"].fillna("未知渠道")

In [14]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "小舞直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a
    

# %%
df4.loc[:,"归属渠道"]=df4.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df4["归属渠道"].value_counts()

归属渠道
芝麻租物                              2998
搜索渠道                              1472
未知渠道                               482
第三方 APP（如钉钉）打开，在跳转链接中传入访问来源参数：      30
京享租右卡                               29
支付宝客户端首页                            21
我的小程序入口                             21
京东618活动                             17
抖音渠道                                13
小程序商家消息（服务提醒）                       11
其他渠道场景渠道。                            7
支付宝 push 消息（同1014）                   6
八派信息                                 3
生活号                                  3
宜品花                                  2
单人聊天会话中的小程序消息卡片（分享）                  1
派金花                                  1
小舞直播                                 1
支付宝学生特惠小程序                           1
Name: count, dtype: int64

In [16]:
df_all = pd.concat([df, df2])
df_all
indx = round(len(df_all)/10)
df_cs = df_all[:indx]
df_xs = df_all[indx:]
df_xs
# 爨超雷	

,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,实际还款日期,应付金额,实付金额,min_date,max_date,diff_date,min_time
82,A20241126171901118,唐文进,18147930888,iPhone 16 Pro Max 全新未激活 国行正品 寄出包邮,2024-11-26 17:19:01,2025-07-29,NaT,695.21,0.0,2025-07-29,2025-07-29,0.0,NaT
83,A20241126173027183,吕鹏,15053522999,iPhone 16 Pro Max 全新未激活 国行正品 寄出包邮,2024-11-26 17:30:27,2025-07-29,NaT,695.21,0.0,2025-07-29,2025-07-29,0.0,NaT
84,A20241126173205190,徐雪怡,15927448411,首月1元 iPhone16ProMax 全新未激活 赠送壳膜 国行正品 顺丰包邮,2024-11-26 17:32:06,2025-07-29,NaT,535.86,0.0,2025-07-29,2025-07-29,0.0,NaT
85,A20241126185510190,李佳慧,18878374851,iPhone15 非监管机 全新国行 顺丰包邮,2024-11-26 18:55:10,2025-07-29,NaT,463.85,0.0,2025-07-29,2025-07-29,0.0,NaT
86,A2024112619283089,尧宁珍,18079453230,iPhone 16 Pro Max 全新未激活 国行正品 寄出包邮,2024-11-26 19:28:30,2025-07-29,NaT,695.21,0.0,2025-07-29,2025-07-29,0.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,A2024072702573862,许凌杰,18772855424,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,2024-07-27 02:57:38,2025-07-30,NaT,729.57,NaN,NaT,NaT,NaN,2025-07-30
123,A20240727135241138,于游,15082000260,iPhone15 非监管机 降价了 全新国行 极速发货,2024-07-27 13:52:42,2025-07-30,NaT,460.69,NaN,NaT,NaT,NaN,2025-07-30
124,A20240727152927128,刘彩云,13615979378,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,2024-07-27 15:29:28,2025-07-30,NaT,1389.10,NaN,NaT,NaT,NaN,2025-07-30
125,A2024072717122952,吴娟,15086851251,iPhone15ProMax 降价了 全新国行 顺丰包邮 低租金高通过,2024-07-27 17:12:29,2025-07-30,NaT,671.21,NaN,NaT,NaT,NaN,2025-07-30


In [42]:
now_date = datetime.now().strftime('%Y-%m-%d')
next_date = (pd.to_datetime(now_date) + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
# df_xs1 = df_xs[df_xs['预计还款日期']==now_date][['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']]
# df_xs2 = df_xs[df_xs['预计还款日期']==next_date][['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']]
# df_xs2
df_xs = df_xs[['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']].sort_values(by='预计还款日期')
df_xs

,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,应付金额
39,A20231004103421142,张春力,13343349833,iPhone14ProMax 国行正品 顺丰包邮 现货速发 高通过率,2023-10-04 10:34:22,2024-10-06,594.64
95,A20240703024618101,吴永强,18085046434,618大促首月1元 iPhone15Pro 全新国行 非监管机 顺丰包邮,2024-07-03 02:46:18,2024-10-06,367.14
6,A2023060422512936,苏东坡,13526466495,iPhone 14 Pro 国行正品 顺丰包邮 现货速发 高通过率,2023-06-04 22:51:30,2024-10-06,726.80
141,A2024080309472068,王超,17736533657,iPhone15 非监管机 降价了 全新国行 极速发货,2024-08-03 09:47:20,2024-10-06,451.59
142,A2024080309564487,方宇涵,18512389125,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,2024-08-03 09:56:45,2024-10-06,457.98
...,...,...,...,...,...,...,...
127,A2024070416133177,张海春,15977477270,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,2024-07-04 16:13:31,2024-10-07,356.15
128,A20240704162741148,易方伟,15888396568,iPhone15ProMax 降价了 全新国行 顺丰包邮 低租金高通过,2024-07-04 16:27:42,2024-10-07,627.83
129,A20240704163026168,王朋勃,13393122220,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,2024-07-04 16:30:27,2024-10-07,356.15
122,A20240704155648282,凌小艳,15751569792,618大促首月1元 iPhone15Pro 全新国行 非监管机 顺丰包邮,2024-07-04 15:56:49,2024-10-07,367.14


In [43]:
# df_cs1 = df_cs[df_cs['预计还款日期']==now_date][['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']]
# df_cs2 = df_cs[df_cs['预计还款日期']==next_date][['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']]
# df_cs1
df_cs = df_cs[['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']].sort_values(by='预计还款日期')
df_cs

,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,应付金额
0,A20231103115127156,杨建明,18655554956,iPhone15 全新国行 24小时内发货 6.1英寸屏幕 顺丰包邮,2023-11-03 11:51:28,2024-10-06,300.61
1,A20231103172547108,王祥,15255610058,iPhone15Pro 全新国行 顺丰包邮,2023-11-03 17:25:48,2024-10-06,600.09
2,A20231103175042205,陈康宁,13489990807,iPhone15ProMax 全新国行 顺丰包邮,2023-11-03 17:50:43,2024-10-06,799.90
19,A2024010318342952,卢少锋,18198231323,全新 Apple Watch Ultra2 智能手表,2024-01-03 18:34:30,2024-10-06,432.66
18,A2024010315415983,叶志乔,13929253979,iPhone15ProMax 全新国行 顺丰包邮,2024-01-03 15:42:00,2024-10-06,666.29
7,A2023120317093112,郭全懂,13266121629,iPhone15ProMax 全新国行 顺丰包邮,2023-12-03 17:09:32,2024-10-06,666.29
8,A2023120323553424,王家宝,18256763788,iPhone14 国行正品 顺丰包邮 现货速发 高通过率,2023-12-03 23:55:34,2024-10-06,327.52
17,A202401031300322,杜琴,18683795663,iPhone15ProMax 全新国行 顺丰包邮,2024-01-03 13:00:33,2024-10-06,632.96
16,A2024010312471177,赵海燕,15708414107,iPhone15ProMax 全新国行 顺丰包邮,2024-01-03 12:47:12,2024-10-06,632.96
21,A2024010412464076,莫锦菲,18565280505,iPhone15ProMax 全新国行 顺丰包邮,2024-01-04 12:46:40,2024-10-07,666.29


In [44]:

with pd.ExcelWriter(f'F:/需求/到期订单/到期订单_信审_{now_date}-{next_date}.xlsx', engine='openpyxl') as writer:
    df_xs.to_excel(writer, sheet_name=now_date, index=False)

In [45]:
# with pd.ExcelWriter(f'F:/需求/到期订单/到期订单_信审_{now_date}-{next_date}.xlsx', engine='openpyxl', mode='a') as writer:
#     df_xs2.to_excel(writer, sheet_name=next_date, index=False)



In [46]:
with pd.ExcelWriter(f'F:/需求/到期订单/到期订单_催收_{now_date}-{next_date}.xlsx', engine='openpyxl') as writer:
    df_cs.to_excel(writer, sheet_name=now_date, index=False)

In [47]:
# with pd.ExcelWriter(f'F:/需求/到期订单/到期订单_催收_{now_date}-{next_date}.xlsx', engine='openpyxl', mode='a') as writer:
#     df_cs2.to_excel(writer, sheet_name=next_date, index=False)

# 逾期订单天数

In [17]:
# df3['逾期天数'] = datetime.now() - df3['预计还款日期']
# df3
df3['逾期天数'] = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))- df3['预计还款日期']
# df3.groupby('逾期天数').agg({'订单号': 'count'})
# df3

# 下单时间上月截止至今日的订单

## 到期订单

In [18]:
df4 = df4[df4['归属渠道']=='八派信息']

In [19]:
dd_dq_month = df4['订单号'].count()
dd_dq_month

np.int64(3)

## 逾期1+订单

In [20]:
df_yq_1= df4[(df4['逾期天数']>=1)]
dd_yq_1 = df_yq_1['订单号'].count()
dd_yq_1

np.int64(0)

## 逾期4+订单

In [21]:
df_yq_4= df4[(df4['逾期天数']>=4)]
dd_yq_4 = df_yq_4['订单号'].count()
dd_yq_4

np.int64(0)

## 逾期7+订单

In [22]:
df_yq_7= df4[(df4['逾期天数']>=7)]
dd_yq_7 = df_yq_7['订单号'].count()
dd_yq_7

np.int64(0)

## 逾期15+订单

In [23]:
df_yq_15= df4[(df4['逾期天数']>=15)]
dd_yq_15 = df_yq_15['订单号'].count()
dd_yq_15

np.int64(0)

# 今日到期订单

In [24]:
Today1 = datetime.now().strftime('%Y-%m-%d')

In [25]:
df_dq= df4[df4['预计还款日期']==Today1]
dd_dq = df_dq['订单号'].count()
df_dq

,channel_name,activity_name,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,实际还款日期,应付金额,实付金额,逾期天数,sort,order_method,来源渠道,归属渠道


## 到期已还款

In [26]:
dq_yhk = df_dq[df_dq['实付金额'] >= df_dq['应付金额']]
dd_dq_yhk = dq_yhk['订单号'].count()
dd_dq_yhk

np.int64(0)

## 到期未还款

In [27]:
dq_whk = df_dq[df_dq['实付金额'] < df_dq['应付金额']]
dd_dq_whk = dq_whk['订单号'].count()
dd_dq_whk

np.int64(0)

# 订单汇总

In [28]:

dd_all = pd.DataFrame({
    '截止日期': [Today1],
    '本月到期订单': [dd_dq_month],
    '今日到期订单': [dd_dq],
    '逾期1+订单': [dd_yq_1],
    '逾期4+订单': [dd_yq_4],
    '逾期7+订单': [dd_yq_7],
    '逾期15+订单': [dd_yq_15],
    '今日到期已还款订单': [dd_dq_yhk],
    '今日到期未还款订单': [dd_dq_whk]
})
dd_all['本月1+逾期率'] = dd_yq_1/dd_dq_month * 100
dd_all['本月1+逾期率'] = dd_all['本月1+逾期率'].apply(lambda x: f'{x: .2f}%')
dd_all['本月4+逾期率'] = dd_yq_4/dd_dq_month * 100
dd_all['本月4+逾期率'] = dd_all['本月4+逾期率'].apply(lambda x: f'{x: .2f}%')
dd_all['本月7+逾期率'] = dd_yq_7/dd_dq_month * 100
dd_all['本月7+逾期率'] = dd_all['本月7+逾期率'].apply(lambda x: f'{x: .2f}%')
dd_all['本月15+逾期率'] = dd_yq_15/dd_dq_month * 100
dd_all['本月15+逾期率'] = dd_all['本月15+逾期率'].apply(lambda x: f'{x: .2f}%')
dd_all['今日到期已还款率'] = dd_dq_yhk/dd_dq * 100
dd_all['今日到期已还款率'] = dd_all['今日到期已还款率'].apply(lambda x: f'{x:.2f}%')
dd_all['今日到期未还款率'] = dd_dq_whk/dd_dq * 100
dd_all['今日到期未还款率'] = dd_all['今日到期未还款率'].apply(lambda x: f'{x:.2f}%')
dd_all = dd_all[['截止日期','本月到期订单','逾期1+订单','本月1+逾期率','逾期4+订单','本月4+逾期率','逾期7+订单','本月7+逾期率','逾期15+订单','本月15+逾期率','今日到期订单','今日到期已还款订单','今日到期已还款率','今日到期未还款订单','今日到期未还款率']]
dd_all

,截止日期,本月到期订单,逾期1+订单,本月1+逾期率,逾期4+订单,本月4+逾期率,逾期7+订单,本月7+逾期率,逾期15+订单,本月15+逾期率,今日到期订单,今日到期已还款订单,今日到期已还款率,今日到期未还款订单,今日到期未还款率
0,2025-07-29,3,0,0.00%,0,0.00%,0,0.00%,0,0.00%,0,0,nan%,0,nan%


In [60]:

with pd.ExcelWriter(f'F:/需求/截止本月到期订单/本月到期订单截止_{Today1}_1.xlsx', engine='openpyxl') as writer:
    dd_all.to_excel(writer, sheet_name='订单汇总', index=False)
with pd.ExcelWriter(f'F:/需求/截止本月到期订单/本月到期订单截止_{Today1}_1.xlsx', engine='openpyxl', mode='a') as writer:
    df4.to_excel(writer, sheet_name='本月到期订单', index=False)
    df_dq.to_excel(writer, sheet_name='今日到期订单', index=False)
    df_yq_1.to_excel(writer, sheet_name='逾期1+订单', index=False)
    df_yq_4.to_excel(writer, sheet_name='逾期4+订单', index=False)
    df_yq_7.to_excel(writer, sheet_name='逾期7+订单', index=False)
    df_yq_15.to_excel(writer, sheet_name='逾期15+订单', index=False)
    dq_yhk.to_excel(writer, sheet_name='今日到期已还款订单', index=False)
    dq_whk.to_excel(writer, sheet_name='今日到期未还款订单', index=False)